# Fitting the superpulse with a double exponential

##### Steps that are taken care of in this code
1. Reads in a list of files corresponding to a period and run.
2. Select an energy region.
3. Pull waveforms corresponding to these energy regimes.
4. Make a natural logarithm of the waveform.
5. Make linear fits corresponfing to the two regions.

Use the legend-sw kernel<br>

Jita (14 Sep 2023)

In [1]:
def energy_selection(hit_df, energy_low, energy_high):
    selection = (hit_df['cuspEmax_ctc_cal'] >= energy_low) & (hit_df['cuspEmax_ctc_cal'] <= energy_high)
    return(selection)

In [2]:
def pileup_flagger(data_raw, evt):
    # Careful with data_raw.values is unsigned. Convert to int before using the np.diff.
    wf_diff = np.diff(data_raw.values[evt].astype(int))
    pileup = np.any(wf_diff[3100:] > 75)
    return(pileup)

In [3]:
def linear(x, a, b):                              
    return a * x + b      

In [4]:
def amp_est(wf):
    wf_max = np.max(wf)
    wf_min = np.min(wf)
    amp = wf_max - wf_min
    return (amp)

In [5]:
def bsln_est(wf):
    bsln = np.average(wf[0:2000])
    return (bsln)

In [6]:
def doubleExp(x, bsln, amp1, amp_sum, tau1, tau2):
    return bsln + amp1 * np.exp(-x/tau1) + (amp_sum - amp1) * np.exp(-x/tau2)

In [7]:
# Import packages
import pygama.lgdo.lh5_store as lh5 
from lgdo import LH5Store
from legendmeta import LegendMetadata

import glob
from datetime import datetime, timezone

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math
from scipy import optimize

In [8]:
# Selecting the data
data_type = "cal"
data_period = "p03"
data_run = "r000"
expt = "l200"

# Setting up the paths
raw_file = sorted(glob.glob(f"/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-orig/archive/raw-v01.00/generated/tier/raw/{data_type}/{data_period}/{data_run}/{expt}-{data_period}-{data_run}-{data_type}-*-tier_raw.lh5"))
dsp_file = sorted(glob.glob(f"/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v01.06/generated/tier/dsp/{data_type}/{data_period}/{data_run}/{expt}-{data_period}-{data_run}-{data_type}-*-tier_dsp.lh5"))
hit_file = sorted(glob.glob(f"/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v01.06/generated/tier/hit/{data_type}/{data_period}/{data_run}/{expt}-{data_period}-{data_run}-{data_type}-*-tier_hit.lh5"))

In [ ]:
i = 0 # Choosing one file

# channel list
ch_list=lh5.ls(raw_file[i])

# extracting the datetime stamp
try:
    dt = raw_file[i][raw_file[i].index("-cal-") + len("-cal-"):raw_file[i].index("-tier_raw.lh5")]
except:
    dt = raw_file[i][raw_file[i].index("-phy-") + len("-phy-"):raw_file[i].index("-tier_raw.lh5")]

# Making a dictionary of germanium detectors
lmeta = LegendMetadata()
chmap = lmeta.hardware.configuration.channelmaps.on(dt)
channel_dict = {}
for channel_name, channel_data in chmap.items():
    if (channel_data.system == "geds"):
        channel_dict[channel_data["daq"]["rawid"]] = (channel_name, channel_data.location.string, channel_data.location.position)

# Choosing one channel
ch = 1104000

# Loop over energies
for emin in range(1500, 2620, 10):

    # initializing the number of good waveforms
    n_wf_acc = 0 
    
    # initializing the number of rejected waveforms
    n_wf_rej = 0

    # Setting up the multiplot
    fig, axs = plt.subplots(nrows = 3, ncols = 3, figsize = (12, 12))
    
    # Pull the hit dataframe
    hit_df = lh5.load_dfs(hit_file[i], ["cuspEmax_ctc_cal","timestamp"], f"ch{ch}/hit")

    # Selecting the energy from the hit dataframe
    emax = emin + 10
    elist = []
    energy_selection1 = energy_selection(hit_df, emin, emax)
    energy_selection1_index = hit_df["cuspEmax_ctc_cal"][energy_selection1].index
    
    # Set a common title for the plot
    #plt.title(f"{channel_dict[ch][0]}, {int(np.average(elist))} keV")

    # Pull out the corresponding waveforms
    store = LH5Store()
    data_raw, n_tot = store.read_object(f"ch{ch}/raw/waveform", raw_file[i], n_rows=np.inf)

    # Initialize the superpulse array
    swf_yval=[0 for x in range(len(data_raw.values[0]))]

    # Go through evt by evt in the energy selection
    for evt in energy_selection1_index:
    
        # Reject the pileup
        pileup = pileup_flagger(data_raw, evt)
        if pileup == False:
            elist.append(hit_df["cuspEmax_ctc_cal"][energy_selection1][evt]) # Maintain a list of energies
            # Plot all the good pulses and keep adding to the superpulse subset
            axs[0,0].plot(np.arange(16*len(data_raw.values[evt]), step = 16), data_raw.values[evt]) # each tick is 16 us, so corrected for that.
            swf_yval=data_raw.values[evt]+swf_yval
            n_wf_acc = n_wf_acc + 1
        else:
            # Plot all the rejected pulses
            axs[0,2].plot(np.arange(16*len(data_raw.values[evt]), step = 16), data_raw.values[evt]) # each tick is 16 us, so corrected for that.
            n_wf_rej = n_wf_rej + 1

    axs[0,0].set_title(f"Accepted pulses")
    axs[0,0].set_xlabel("Time (ns)")
    axs[0,0].set_ylabel("ADC")
    axs[0,0].text(0.2, 0.9, f"n_wf = {n_wf_acc}", horizontalalignment='center', verticalalignment='center', transform=axs[0,0].transAxes)

    # Taking care of the normalization
    swf_yval = swf_yval/n_wf_acc

    # Generating the timestamps, considering that one tick is 16 us
    swf_xval = np.arange(16*len(data_raw.values[evt]), step = 16)

    # Amplitude and baseline estimators
    amp = amp_est(swf_yval)
    bsln = bsln_est(swf_yval)
    
    # Plotting the superpulse
    axs[0,1].plot(swf_xval, swf_yval) # each tick is 16 us, so corrected for that.
    axs[0,1].set_title(f"Superpulse (accepted pulses)")
    axs[0,1].set_xlabel("Time (ns)")
    axs[0,1].set_ylabel("ln(ADC)")
    
    # Setting up plot for rejected pulses
    axs[0,2].set_title(f"Rejected pulses")
    axs[0,2].set_xlabel("Time (ns)")
    axs[0,2].set_ylabel("ADC")
    axs[0,2].axvline(3200*16, color = "red")
    axs[0,2].text(0.2, 0.9, f"n_wf = {n_wf_rej}", horizontalalignment='center', verticalalignment='center', transform=axs[0,2].transAxes)

    # Linear fits
    
    # Generating the natural log of the superpulse to get estimates
    lswf_yval=[]
    for x in swf_yval:
        lswf_yval.append(math.log(x))

    # data for the linear fit in the fast tau region
    xval_t1=swf_xval[3200:4000]
    yval_t1= lswf_yval[3200:4000]

    # data for the linear fit in the slow tau region
    xval_t2=swf_xval[-1000:]
    yval_t2= lswf_yval[-1000:]

    # linear fit for the fast tau region
    initialguess = [0, 10]
    fit_t1, covariance_t1 = optimize.curve_fit(           
            linear,                                     
            xval_t1,   
            yval_t1,    
            initialguess)  
    fit_t1_err = np.sqrt(np.diag(covariance_t1)) # From scipy documentation

    print(f" Linear Fit 1: \n \n SLOPE \n Guess: {initialguess[0]}, \n Fit: {fit_t1[0]}, \n Error: {fit_t1_err[0]}, \n \n INTERCEPT \n Guess: {initialguess[1]}, \n Fit: {fit_t1[1]}, \n Error: {fit_t1_err[1]} \n ----------")

    # linear fit for the slow region
    fit_t2, covariance_t2 = optimize.curve_fit(           
            linear,                                     
            xval_t2,   
            yval_t2,    
            initialguess)  
    fit_t2_err = np.sqrt(np.diag(covariance_t2)) # From scipy documentation

    print(f" Linear Fit 2: \n \n SLOPE \n Guess: {initialguess[0]}, \n Fit: {fit_t2[0]}, \n Error: {fit_t2_err[0]}, \n \n INTERCEPT \n Guess: {initialguess[1]}, \n Fit: {fit_t2[1]}, \n Error: {fit_t2_err[1]} \n ----------")
    
    # Generate the fitted lines for plotting
    x_fit_t1 = np.linspace(50000,130000, 100)
    y_fit_t1 = linear(x_fit_t1, *fit_t1)

    x_fit_t2 = np.linspace(50000,130000, 100)
    y_fit_t2 = linear(x_fit_t2, *fit_t2)

    # Plot the linearized function and fits
    axs[1,0].plot(swf_xval,lswf_yval)
    axs[1,0].plot(x_fit_t1, y_fit_t1, c = "red", linestyle = "dashed", label = f"fit1 \n tau1_ns = {-1/fit_t1[0]:.3e} \n")
    axs[1,0].plot(x_fit_t2, y_fit_t2, c = "orange", linestyle = "dashed", label = f"fit2 \n tau2_ns = {-1/fit_t2[0]:.3e} \n")
    axs[1,0].set_title(f"Natural log of superpulse")
    axs[1,0].set_xlabel("Time (ns)")
    axs[1,0].set_ylabel("ADC")
    axs[1,0].legend()
    
    # Fitting superpulse to the double exponential
    initialguess = [bsln, amp/2, amp, -1/fit_t1[0], -1/fit_t2[0]]
    #initialguess = [bsln, 0, amp, 0, -1/fit_t2[0]]
    bounds = ([bsln-0.1, 0, 0, 0, 0], [bsln+0.1, amp, np.inf, np.inf, np.inf]) # force all parameters positive
    expfit, expfit_covariance = optimize.curve_fit(           
            doubleExp,                                     
            swf_xval[3200:],   
            swf_yval[3200:],    
            initialguess,
            bounds = bounds)  

    expfit_err = np.sqrt(np.diag(expfit_covariance)) # From scipy documentation
    print(f" Expfit: \n \n BASELINE \n Guess: {initialguess[0]}, \n Fit: {expfit[0]}, \n Error: {expfit_err[0]}, \n \n AMPLITUDE1 \n Guess: {initialguess[1]}, \n Fit: {expfit[1]}, \n Error: {expfit_err[1]}, \n \n AMPLITUDE \n Guess: {initialguess[2]}, \n Fit: {expfit[2]}, \n Error: {expfit_err[2]}, \n \n TAU1 \n Guess: {initialguess[3]}, \n Fit: {expfit[3]}, \n Error: {expfit_err[3]}, \n \n TAU2 \n Guess: {initialguess[4]}, \n Fit: {expfit[4]}, \n Error: {expfit_err[4]} \n ----------")
    
    # Generate a fitted double exp line for plotting
    expfit_y = doubleExp(swf_xval[3200:], *expfit)

    # Plot the superpulse and the doubleExp fitted line
    axs[2,0].plot(swf_xval,swf_yval)
    axs[2,0].plot(swf_xval[3200:], expfit_y, c="red", linestyle='dashed')
    axs[2,0].set_title(f"Superpulse fits")
    axs[2,0].set_xlabel("Time (ns)")
    axs[2,0].set_ylabel("ADC")
    axs[2,0].plot(swf_xval, swf_yval, c = "cyan" , label = "data \n")
    axs[2,0].plot(swf_xval[3200:], expfit_y, c = "red", linestyle = 'solid', label = f"fit \n tau1_ns = {expfit[3]:.3e} \n tau2_ns = {expfit[4]:.3e} \n")
    axs[2,0].legend()
    
    # Plot the residuals
    axs[2,1].plot(swf_yval[3200:], swf_yval[3200:]-expfit_y, c="red")
    axs[2,1].set_title(f"Residuals")
    axs[2,1].set_xlabel("Time (ns)")
    axs[2,1].set_ylabel("ADC")
    
    # Format the figure globally and save
    plt.tight_layout()
    plt.savefig(f"{channel_dict[ch][0]}_{int(np.average(elist))}.png")

 Linear Fit 1: 
 
 SLOPE 
 Guess: 0, 
 Fit: -9.466013912326184e-07, 
 Error: 5.4767693092092765e-09, 
 
 INTERCEPT 
 Guess: 10, 
 Fit: 10.162983374521708, 
 Error: 0.00031606661491804107 
 ----------
 Linear Fit 2: 
 
 SLOPE 
 Guess: 0, 
 Fit: -6.60907105739451e-07, 
 Error: 3.864752236192767e-09, 
 
 INTERCEPT 
 Guess: 10, 
 Fit: 10.140955002226606, 
 Error: 0.00047594673150498347 
 ----------
 Expfit: 
 
 BASELINE 
 Guess: 15002.396, 
 Fit: 15002.49599999968, 
 Error: 2808.3919461376736, 
 
 AMPLITUDE1 
 Guess: 4893.75, 
 Fit: 2255.5200380513916, 
 Error: 419.18490387331093, 
 
 AMPLITUDE 
 Guess: 9787.5, 
 Fit: 12743.636940793927, 
 Error: 2343.6925003998626, 
 
 TAU1 
 Guess: 1056410.8707867507, 
 Fit: 18892.30502659958, 
 Error: 2834.8367405015256, 
 
 TAU2 
 Guess: 1513071.9450824445, 
 Fit: 546378.6338944254, 
 Error: 185395.5264533821 
 ----------
 Linear Fit 1: 
 
 SLOPE 
 Guess: 0, 
 Fit: -9.474139916316793e-07, 
 Error: 2.737478581300641e-09, 
 
 INTERCEPT 
 Guess: 10, 
 Fit

/tmp/ipykernel_983100/3289223201.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(nrows = 3, ncols = 3, figsize = (12, 12))


 Linear Fit 1: 
 
 SLOPE 
 Guess: 0, 
 Fit: -1.0425352467468128e-06, 
 Error: 3.4510116794470907e-09, 
 
 INTERCEPT 
 Guess: 10, 
 Fit: 10.22688797157435, 
 Error: 0.00019915930829330955 
 ----------
 Linear Fit 2: 
 
 SLOPE 
 Guess: 0, 
 Fit: -7.295074819611745e-07, 
 Error: 2.40915856524845e-09, 
 
 INTERCEPT 
 Guess: 10, 
 Fit: 10.203173934684832, 
 Error: 0.0002966894352625276 
 ----------
 Expfit: 
 
 BASELINE 
 Guess: 15227.121, 
 Fit: 15227.220912846753, 
 Error: 1329.1198171427197, 
 
 AMPLITUDE1 
 Guess: 5535.25, 
 Fit: 3681.2211146772324, 
 Error: 525.2135820288054, 
 
 AMPLITUDE 
 Guess: 11070.5, 
 Fit: 15622.18050223123, 
 Error: 843.0111373125729, 
 
 TAU1 
 Guess: 959200.1835146176, 
 Fit: 15907.629752853723, 
 Error: 1167.5740013103564, 
 
 TAU2 
 Guess: 1370787.85992934, 
 Fit: 523981.6192231644, 
 Error: 73786.90834727668 
 ----------
 Linear Fit 1: 
 
 SLOPE 
 Guess: 0, 
 Fit: -1.030209523057351e-06, 
 Error: 7.648440198830126e-09, 
 
 INTERCEPT 
 Guess: 10, 
 Fit: 10